In [4]:
import pandas as pd
import spacy
import os
import json
import operator
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *
porter_stemmer = PorterStemmer()
import readability
import vaderSentiment
#import sys 
#!{sys.executable} -m pip install vadersentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
from readability import Readability  #https://github.com/cdimascio/py-readability-metrics
stop_words = nltk.corpus.stopwords.words('english')
#from profanity_check import predict  # https://github.com/vzhou842/profanity-check
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
nlp = spacy.load('C://Users//seanp//Anaconda3//Lib//site-packages//en_core_web_sm//en_core_web_sm-2.0.0')


main_dict_path = os.getcwd()[:os.getcwd().find("feat_extraction")]

movies = pd.read_csv(main_dict_path + "success_data.csv", index_col=0)
movies["Success"] = movies.apply(lambda row: 1 if row["Worldwide ROI (%)"] > 0 else 0, axis=1)
movies.drop(movies.columns[1:7], axis=1, inplace=True)
movies.head()

,Processed Title,Success
Avatar,Avatar,1
The Dark Knight Rises,"Dark-Knight-Rises,-The",1
The Avengers,"Avengers,-The",1
Pirates of the Caribbean: Dead Man's Chest,Pirates-of-the-Caribbean-Dead-Man's-Chest,1
Men in Black 3,Men-in-Black-3,1


In [5]:
path = main_dict_path + "diag_jsons/"
n_unique_words_dict = {"n_unique_words_char_1": [], "n_unique_words_char_2": [],
                      "n_unique_words_char_3": [], "n_unique_words_char_4": [],
                      "n_unique_words_char_5": []}
read_level_dict = {"FK_read_level_char_1": [], "FK_read_level_char_2": [],
                   "FK_read_level_char_3": [], "FK_read_level_char_4": [],
                   "FK_read_level_char_5": []}
n_stop_words = {"n_stop_words_char_1": [], "n_stop_words_char_2": [],
               "n_stop_words_char_3": [], "n_stop_words_char_4": [],
               "n_stop_words_char_5": []}
n_curse_words = {"n_curse_words_char_1": [], "n_curse_words_char_2": [],
               "n_curse_words_char_3": [], "n_curse_words_char_4": [],
               "n_curse_words_char_5": []}
overall_polarity = {"overall_polarity_char_1":[],"overall_polarity_char_2":[],
                 "overall_polarity_char_3":[],"overall_polarity_char_4":[],
                 "overall_polarity_char_5":[]}
num_pass_sents = {"num_pass_sents_char_1":[],"num_pass_sents_char_2":[],
                 "num_pass_sents_char_3":[],"num_pass_sents_char_4":[],
                 "num_pass_sents_char_5":[]}
num_sents = {"num_sents_char_1":[],"num_sents_char_2":[],
                 "num_sents_char_3":[],"num_sents_char_4":[],
                 "num_sents_char_5":[]}

prob = []
for script in movies["Processed Title"]:
    
    with open(path + script + "_script.json") as handle:
        char_diags = json.loads(handle.read())
        
    # Gets the dictionary for only the 5 characters with the most dialogue
    dict_lengths = {}
    for key, value in char_diags["dialogues"].items():
        a = value.replace("\n", "").replace(" ", "")
        i = 1
        while True:
            before = len(a)
            a = a.replace("[" + str(i) + "]", "")
            i += 1
            if len(a) == before:
                break
        dict_lengths[key] = len(a)
    top_chars = [char[0] for char in sorted(dict_lengths.items(), key=operator.itemgetter(1), reverse=True)[:5]]
    all_chars = list(char_diags["dialogues"].keys())
    # so I can run this and go over the top 5 characters in every movie...... 
    # or I THINK just get rid of the for char in all chars part... 
    for char in all_chars:
        if char not in top_chars:
            del char_diags["dialogues"][char]
    
    # Loops over the characters dialogues to extract some features (per character)
    n_char = 1
    for dialogue in char_diags["dialogues"].values():
        # Gets the dialogue in one long sentence and without [i]
        diag = dialogue.replace("\n", "")
        i = 1
        while True:
            before = len(diag)
            diag = diag.replace("[" + str(i) + "]", "")
            i += 1
            if len(diag) == before:
                break
        
        # FEAT #1: # Number of Unique words per character --> Numerical
        # Tokenizes the long sentence, actually, pedro I think it's still a string because the punctuation still exists!
        words = word_tokenize(diag)
        # segment the sentences 
        sentz = sentence_tokenizer.tokenize(diag)
        # iterate though each string inside of the list of sentences       
        num_passive = 0
        for sentence in sentz:
            # the nlp was essentially using spacy's 'en' module on the sentence, which is commented out above 
            # because I had the worst time actually importing it from python without using the location of the module
            tagged_sent = nlp(sentence)
            # checking if any of the words in the sentence are tagged as being what spacy considers to be 'passive'
            passive_toks=[tok for tok in tagged_sent if (tok.dep_ == "nsubjpass" or tok.dep_ =="auxpass")]
            # this means we will add 1 to our variable that contains the number of passive sentences... 
            if passive_toks != []:
                num_passive +=1
        # this gets the percent of passive sentences. 
         
        
        
        num_pass_sents["num_pass_sents_char_" + str(n_char)].append(num_passive)
        num_sents["num_sents_char_" + str(n_char)].append(len(sentz))
                    
        
        # Stems the tokens
        punct_sings = ".!?,;:-_--\'\'\'``..."
        words_stemmed = [porter_stemmer.stem(word) for word in words if word not in punct_sings]
        # Gets the number of unique stems
        n_unique_words_dict["n_unique_words_char_" + str(n_char)].append(len(set(words_stemmed)))
        
        # FEAT #2: Flesch Kincaid Read Level per character --> Categorical
        # https://github.com/cdimascio/py-readability-metrics
        r = Readability(diag)
        read_level_dict["FK_read_level_char_" + str(n_char)].append(r.flesch_kincaid().grade_level)
        prob.append(script)
        
        # FEAT #3: Number of Stop Words per character --> Numerical
        n_stop_words["n_stop_words_char_" + str(n_char)].append(len(set([w for w in words if w.lower() in stop_words])))
        
        
        # FEAT #4: Number of curse words per character -> Numerical
        # https://github.com/vzhou842/profanity-check
        #n_curse = 0
        #for word in words:
        #    if predict([word]): 
        #        n_curse += 1
        #n_curse_words["n_curse_words_char_" + str(n_char)].append(n_curse)
        
        # Feat 5: Overall Polarity of Character's dialogue --> Numerical
        overall_polarity["overall_polarity_char_" + str(n_char)].append(analyser.polarity_scores(diag)['compound'])
        # I can easily break this into overall positive and overall negative as well..... 
        # and they are here, but just note... this takes A WHILE to run... 
        
        # up 
        
        
        
        n_char += 1

In [6]:
def append_feature(feature_dict):
    for key in feature_dict.keys():
        movies[key] = feature_dict[key]
        
append_feature(n_unique_words_dict)
append_feature(read_level_dict)
append_feature(n_stop_words)
append_feature(overall_polarity)
append_feature(num_pass_sents)
append_feature(num_sents)

# we will essentially get the total passive sentences words across ALLLL CHARACTERS for a movie 
# and divide by the total sentneces each of them speak 
for index, row in feat_frame.iterrows():
    



In [7]:
movies.head()

,Processed Title,Success,n_unique_words_char_1,n_unique_words_char_2,n_unique_words_char_3,n_unique_words_char_4,n_unique_words_char_5,FK_read_level_char_1,FK_read_level_char_2,FK_read_level_char_3,...,num_pass_sents_char_1,num_pass_sents_char_2,num_pass_sents_char_3,num_pass_sents_char_4,num_pass_sents_char_5,num_sents_char_1,num_sents_char_2,num_sents_char_3,num_sents_char_4,num_sents_char_5
Avatar,Avatar,1,670,565,252,425,276,2,2,2,...,13,8,5,2,3,428,227,127,132,74
The Dark Knight Rises,"Dark-Knight-Rises,-The",1,531,514,506,418,434,2,2,3,...,13,6,10,8,5,265,226,169,149,159
The Avengers,"Avengers,-The",1,560,623,359,425,205,2,3,3,...,11,7,3,4,3,286,319,122,133,55
Pirates of the Caribbean: Dead Man's Chest,Pirates-of-the-Caribbean-Dead-Man's-Chest,1,629,373,285,378,281,2,1,1,...,13,6,6,5,3,331,207,134,132,73
Men in Black 3,Men-in-Black-3,1,1206,867,234,209,185,2,3,2,...,18,12,2,2,3,732,357,60,56,38


In [5]:
movies.to_csv("movies_with_feats_sean.csv")

In [20]:
movies['tot_pass_sents'] = movies.num_pass_sents_char_1+movies.num_pass_sents_char_2+movies.num_pass_sents_char_3+movies.num_pass_sents_char_4+movies.num_pass_sents_char_5
    # ok so lets' try to extract something else!!! 
# 
movies['tot_sents'] = movies.num_sents_char_1+movies.num_sents_char_2+movies.num_sents_char_3+movies.num_sents_char_4+movies.num_sents_char_5
movies['passive_ratio'] = movies.tot_pass_sents/movies.tot_sents

In [21]:
movies.head()

,Processed Title,Success,n_unique_words_char_1,n_unique_words_char_2,n_unique_words_char_3,n_unique_words_char_4,n_unique_words_char_5,FK_read_level_char_1,FK_read_level_char_2,FK_read_level_char_3,...,num_pass_sents_char_4,num_pass_sents_char_5,num_sents_char_1,num_sents_char_2,num_sents_char_3,num_sents_char_4,num_sents_char_5,tot_sents,tot_pass_sents,passive_ratio
Avatar,Avatar,1,670,565,252,425,276,2,2,2,...,2,3,428,227,127,132,74,988,31,0.031377
The Dark Knight Rises,"Dark-Knight-Rises,-The",1,531,514,506,418,434,2,2,3,...,8,5,265,226,169,149,159,968,42,0.043388
The Avengers,"Avengers,-The",1,560,623,359,425,205,2,3,3,...,4,3,286,319,122,133,55,915,28,0.030601
Pirates of the Caribbean: Dead Man's Chest,Pirates-of-the-Caribbean-Dead-Man's-Chest,1,629,373,285,378,281,2,1,1,...,5,3,331,207,134,132,73,877,33,0.037628
Men in Black 3,Men-in-Black-3,1,1206,867,234,209,185,2,3,2,...,2,3,732,357,60,56,38,1243,37,0.029767


In [24]:
movies = movies.drop(['num_pass_sents_char_1','num_pass_sents_char_2','num_pass_sents_char_3','num_pass_sents_char_4','num_pass_sents_char_5','num_sents_char_1','num_sents_char_2','num_sents_char_3','num_sents_char_4','num_sents_char_5'],axis=1)

In [25]:
movies.head()

,Processed Title,Success,n_unique_words_char_1,n_unique_words_char_2,n_unique_words_char_3,n_unique_words_char_4,n_unique_words_char_5,FK_read_level_char_1,FK_read_level_char_2,FK_read_level_char_3,...,n_stop_words_char_4,n_stop_words_char_5,overall_polarity_char_1,overall_polarity_char_2,overall_polarity_char_3,overall_polarity_char_4,overall_polarity_char_5,tot_sents,tot_pass_sents,passive_ratio
Avatar,Avatar,1,670,565,252,425,276,2,2,2,...,101,94,0.9919,0.9946,0.9861,0.7676,-0.9982,988,31,0.031377
The Dark Knight Rises,"Dark-Knight-Rises,-The",1,531,514,506,418,434,2,2,3,...,107,121,0.9871,-0.9975,0.8331,-0.9751,-0.9954,968,42,0.043388
The Avengers,"Avengers,-The",1,560,623,359,425,205,2,3,3,...,96,68,0.9704,0.9790,0.9189,-0.9782,0.9739,915,28,0.030601
Pirates of the Caribbean: Dead Man's Chest,Pirates-of-the-Caribbean-Dead-Man's-Chest,1,629,373,285,378,281,2,1,1,...,103,86,0.9980,-0.9658,0.9959,0.9297,0.9926,877,33,0.037628
Men in Black 3,Men-in-Black-3,1,1206,867,234,209,185,2,3,2,...,79,67,0.9979,0.9839,0.9903,-0.9787,0.9617,1243,37,0.029767
